<a href="https://colab.research.google.com/github/Biswamitra4599/MRI_Literature_Review/blob/main/Copy_of_Bart_sense_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#USING BART


## Importing System Libraries & Mounting Drive


In [1]:
import os
import sys
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image
import h5py
import numpy as np
import glob

In [2]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Remove an empty directory
mri_path="/content/drive/MyDrive/MRI/"
cfl_path=mri_path+"CFL/"
buffer_path=mri_path+"BUFFER/"
h5_path=mri_path+"H5/"
test_path=mri_path+"TEST/"

try:
    os.rmdir(buffer_path)
    print(f"Directory '{buffer_path}' removed successfully.")
except OSError as e:
    print(f"Error: {e.strerror}")

Directory '/content/drive/MyDrive/MRI/BUFFER/' removed successfully.


In [5]:
# Managing the Paths to avoid any Error
if not os.path.exists(mri_path):
  os.makedirs(mri_path)
  os.makedirs(cfl_path)
  os.makedirs(buffer_path)
  os.makedirs(h5_path)
  os.makedirs(test_path)
  print(f"Path Created!!! Put the '.h5' files in the specific place: {h5_path}")
else:
  os.makedirs(buffer_path)
  print("Path Already Exists!!")


Path Already Exists!!


## Fast MRI Setup

In [6]:
!pip install fastmri

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 48.5 MB/s eta 0:00:00
  Created wheel for runstats: filename=runstats-2.0.0-py3-none-any.whl size=12360 sha256=24fa688eaf4c74e54a22c5f5351e2c46806fbd3d9b1eece571cb6ce77bed1630
  Stored in directory: /root/.cache/pip/wheels/86/c9/78/84378178c355b0044253b843a5ae471e6b15f4a4841058de1f
Successfully built runstats


In [7]:
import fastmri
from fastmri.data import transforms as T

## Sense Reconstruction
by: Mahesh (SIR)

### Setting up the GPU and the appt environment

In [8]:
%%bash

# Use CUDA 10.1 when on Tesla K80

# Estimate GPU Type
GPU_NAME=$(nvidia-smi --query-gpu=gpu_name --format=csv,noheader)

echo "GPU Type:"
echo $GPU_NAME

if [ "Tesla K80" = "$GPU_NAME" ];
then
    echo "GPU type Tesla K80 does not support CUDA 11. Set CUDA to version 10.1."

    # Change default CUDA to version 10.1
    cd /usr/local
    rm cuda
    ln -s cuda-10.1 cuda
    echo "done!!!"
else
    echo "Current GPU supports default CUDA-11."
    echo "No further actions are necessary."
fi

echo "GPU Information:"
# nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
# nvcc --version

GPU Type:

Current GPU supports default CUDA-11.
No further actions are necessary.
GPU Information:


bash: line 5: nvidia-smi: command not found


### BART INSTALLATION

1.  Dependencies for BART,
2.  Installation of BART,
3.  Environment Setup.







In [9]:
%%bash

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Clone Bart
[ -d /content/bart ] && rm -r /content/bart
git clone https://github.com/mrirecon/bart/ bart &> /dev/null
echo "Dependencies Done!!"
pwd

Dependencies Done!!
/content


In [10]:
%%bash
cd bart
git branch -a


* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/altmin
  remotes/origin/delayed_and_distributed_computing
  remotes/origin/fft-flags
  remotes/origin/file-format
  remotes/origin/fix_ismrmrd_distribution_package
  remotes/origin/gh-pages
  remotes/origin/intel_kernels
  remotes/origin/low_mem
  remotes/origin/master
  remotes/origin/master_precond
  remotes/origin/md_zabs
  remotes/origin/motion
  remotes/origin/mpi
  remotes/origin/na
  remotes/origin/nn_ismrm2021
  remotes/origin/no_capture
  remotes/origin/pipes
  remotes/origin/reconet_delayed
  remotes/origin/reconet_delayed2
  remotes/origin/reconet_kspace
  remotes/origin/sms-nlinv
  remotes/origin/steepest-descent
  remotes/origin/wavecaipi


In [15]:
%%bash
sudo apt-get update
sudo apt-get install -y gcc-12 g++-12
gcc-12 --version
g++-12 --version


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (3,316 B/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
gcc-12 is already the newest version (12.3.0-1ubuntu1~22.04.2).
g++-12 is already the newest version (12.3.0-1ubuntu1~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
gcc-12 (Ubuntu 12.3.0-1ubuntu1~22.04.2) 12.3.0
Copyright (C) 2022 F

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [17]:
%%bash
cd bart || exit 1

echo "PARALLEL=1
CUDA=1
CUDA_BASE=/usr/local/cuda
CUDA_LIB=lib64
OPENBLAS=1
BLAS_THREADSAFE=1
CC=gcc-12
CXX=g++-12" > Makefiles/Makefile.local



In [18]:
%%bash
cd bart || exit 1
make clean
make


CONFIGURATION MODIFIED. RUNNING FULL REBUILD.
echo "#define MAIN_LIST affinereg, avg, bench, bin, bitmask, bloch, cabs, calc, caldir, calmat, carg, casorati, cc, ccapply, cdf97, circshift, coils, compress, conj, conv, conway, copy, cpyphs, creal, crop, cunet, delta, denoise, ecalib, ecaltwo, epg, estdelay, estdims, estmotion, estscaling, estshift, estvar, extract, fakeksp, fft, fftmod, fftrot, fftshift, filter, flatten, flip, fmac, fovshift, gmm, grid, grog, hist, homodyne, ictv, index, interpolate, invert, itsense, join, looklocker, lrmatrix, mandelbrot, measure, mip, mnist, moba, mobafit, morphop, multicfl, ncalib, nlinv, nlinvnet, nlmeans, nnet, noise, normalize, nrmse, nufft, nufftbase, onehotenc, ones, pattern, phantom, phasepole, pics, pocsense, poisson, pol2mask, poly, psf, pulse, raga, reconet, repmat, reshape, resize, rmfreq, rof, roistat, rovir, rss, rtnlinv, sake, sample, saxpy, scale, sdot, seq, show, signal, sim, slice, spow, sqpics, squeeze, ssa, std, stl, svd, tee, tenso

In file included from /content/bart/src/bart.c:49:
/content/bart/src/num/gpuops.h:6:10: fatal error: cuda_runtime_api.h: No such file or directory
    6 | #include <cuda_runtime_api.h>
      |          ^~~~~~~~~~~~~~~~~~~~
compilation terminated.
make: *** [Makefile:981: /content/bart/src/bart.o] Error 1
make: *** Waiting for unfinished jobs....
In file included from src/mnist.c:24:
src/num/gpuops.h:6:10: fatal error: cuda_runtime_api.h: No such file or directory
    6 | #include <cuda_runtime_api.h>
      |          ^~~~~~~~~~~~~~~~~~~~
compilation terminated.
make: *** [Makefile:981: src/mnist.o] Error 1
In file included from src/nnet.c:22:
src/num/gpuops.h:6:10: fatal error: cuda_runtime_api.h: No such file or directory
    6 | #include <cuda_runtime_api.h>
      |          ^~~~~~~~~~~~~~~~~~~~
compilation terminated.
make: *** [Makefile:981: src/nnet.o] Error 1
In file included from src/moba.c:45:
src/num/gpuops.h:6:10: fatal error: cuda_runtime_api.h: No such file or directory
   

CalledProcessError: Command 'b'cd bart || exit 1\nmake clean\nmake\n'' returned non-zero exit status 2.

In [ ]:
# %%bash
# pwd
# BRANCH=ISMRM21

# cd bart || exit 1

# pwd
# # Switch to desired branch of the BART project
# git checkout -q $BRANCH

# pwd

# # Define specifications
# COMPILE_SPECS="PARALLEL=1 CUDA=1 CUDA_BASE=/usr/local/cuda CUDA_LIB=lib64 OPENBLAS=1 BLAS_THREADSAFE=1"

# printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local

# make &> /dev/null


In [ ]:
# Define environment variables for BART and OpenMP
os.environ['TOOLBOX_PATH'] = "/content/bart"
os.environ['OMP_NUM_THREADS']="4"
# Add the BARTs toolbox to the PATH variable
os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append("/content/bart/python")
print(sys.path)

['/content', '/env/python', '/usr/lib/python312.zip', '/usr/lib/python3.12', '/usr/lib/python3.12/lib-dynload', '', '/usr/local/lib/python3.12/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.12/dist-packages/IPython/extensions', '/root/.ipython', '/content/bart/python']


In [ ]:

!echo "# The BART used in this notebook:"
!cd bart/
!which bart
!echo "# BART version: "
!bart version
!bart


# The BART used in this notebook:
/content/bart/bart
# BART version: 
v0.7.00-88-g9d66c09
BART. Available commands are:
avg         bench       bin         bitmask     cabs        caldir      
calmat      carg        casorati    cc          ccapply     cdf97       
circshift   conj        conv        conway      copy        cpyphs      
creal       crop        delta       ecalib      ecaltwo     epg         
estdelay    estdims     estshift    estvar      extract     fakeksp     
fft         fftmod      fftrot      fftshift    filter      flatten     
flip        fmac        homodyne    index       invert      itsense     
join        looklocker  lrmatrix    mandelbrot  mip         moba        
mobafit     nlinv       noise       normalize   nrmse       nufft       
ones        pattern     phantom     pics        pocsense    poisson     
pol2mask    poly        repmat      reshape     resize      rmfreq      
rof         roistat     rss         rtnlinv     sake        saxpy       
scal

## Moving into the Problem

### Remaining Libs

In [ ]:
# More python libraries
import cfl

### Setup Visualization Helper
https://github.com/mrirecon/bart/blob/master/python/cfl.py

https://colab.research.google.com/github/mrirecon/bart-workshop/blob/master/mri_together_2021/bart_intro_examples.ipynb#scrollTo=__DLJ4XLDwXx


In [ ]:
def plot_map(dataname, colorbar, vmin, vmax, cbar_label):

    # Import data
    data = np.abs(cfl.readcfl(dataname).squeeze())

    # Import colorbar type
    colorbartype =  colorbar

    # Set zero to a black color for a masking effect
    my_cmap = cm.get_cmap(colorbartype, 256)
    my_cmap.set_bad('black')

    data = np.ma.masked_equal(data, 0)

    fig = plt.figure(figsize=(15,15))
    ax1 = fig.add_subplot(111)
    im = ax1.imshow(data, interpolation='nearest', cmap=my_cmap, vmin=vmin, vmax=vmax)

    # Style settings
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_label(cbar_label)
    cbar.ax.tick_params()

    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    ax1.xaxis.set_ticks_position('none')
    ax1.yaxis.set_ticks_position('none')
    ax1.set_axis_off()

    plt.show()

### Getting Data For Visualization

In [ ]:
import os
import urllib.request
import zipfile

# url_path = "https://github.com/mrirecon/bart-workshop/blob/master/mri_together_2021/data.zip"
dest_path = "/content/drive/MyDrive/MRI/H5"
zip_file_name = "/content/drive/MyDrive/MRI/H5/data.zip"

# # Check if the file already exists
# if not os.path.isfile(zip_file_name):
#     # Download the file using urllib
#     print(f"Downloading {zip_file_name} from {url_path}...")
#     urllib.request.urlretrieve(url_path, zip_file_name)

# Check if the downloaded file is a ZIP file
if zip_file_name.endswith('.zip'):
    # Unzip the file if it's a valid ZIP file
    if os.path.isfile(zip_file_name):
        print(f"Unzipping {zip_file_name}...")
        with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
            zip_ref.extractall(dest_path)
    else:
        print(f"{zip_file_name} not found.")
else:
    print(f"{zip_file_name} is not a valid ZIP file.")


Unzipping /content/drive/MyDrive/MRI/H5/data.zip...


BadZipFile: File is not a zip file

In [ ]:
# %%bash
# url_path= "https://github.com/mrirecon/bart-workshop/blob/master/mri_together_2021/data.zip"
# dest_path= "/content/drive/MyDrive/MRI/H5"
# [ ! -f data.zip ] && wget -q $url_path -o $dest_path
# unzip -n data.zip

# #https://github.com/mrirecon/bart-workshop/blob/master/mri_together_2021/data.zip

bash: line 1: https://github.com/mrirecon/bart-workshop/blob/master/mri_together_2021/data.zip: No such file or directory
bash: line 2: /content/drive/MyDrive/MRI/H5: Is a directory
wget: option requires an argument -- 'o'
Usage: wget [OPTION]... [URL]...

Try `wget --help' for more options.
unzip:  cannot find or open data.zip, data.zip.zip or data.zip.ZIP.


CalledProcessError: Command 'b'url_path= "https://github.com/mrirecon/bart-workshop/blob/master/mri_together_2021/data.zip"\ndest_path= "/content/drive/MyDrive/MRI/H5"\n[ ! -f data.zip ] && wget -q $url_path -o $dest_path\nunzip -n data.zip\n\n#https://github.com/mrirecon/bart-workshop/blob/master/mri_together_2021/data.zip\n'' returned non-zero exit status 9.

In [ ]:
# !unzip data.zip -d /content/data

In [ ]:
! echo $'\n# K-Space (fully sampled):'
! head -n2 /content/data/data/kspace_fs.hdr

! echo $'\n# Pattern:'
! head -n2 /content/data/data/pattern_po_4.hdr

pattern = np.abs(cfl.readcfl("/content/data/data/pattern_po_4")) # 640 368 1 1 1
plt.imshow(pattern, cmap="gray")
plt.show()

# Multiplying to the mask
! bart fmac /content/data/data/kspace_fs /content/data/data/pattern_po_4 /content/data/kspace_ds


# K-Space (fully sampled):
head: cannot open '/content/data/data/kspace_fs.hdr' for reading: No such file or directory

# Pattern:
head: cannot open '/content/data/data/pattern_po_4.hdr' for reading: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/data/pattern_po_4.hdr'

In [ ]:
!bart ecalib /content/data/kspace_ds sensitivities
!bart pics -l1 -r0.001 /content/data/kspace_ds sensitivities image_out2


Loading cfl file /content/data/kspace_ds
 : No such file or directory
ERROR: Loading cfl file /content/data/kspace_ds
 : No such file or directory
ERROR: 

In [ ]:
!bart


BART. Available commands are:
avg         bench       bin         bitmask     cabs        caldir      
calmat      carg        casorati    cc          ccapply     cdf97       
circshift   conj        conv        conway      copy        cpyphs      
creal       crop        delta       ecalib      ecaltwo     epg         
estdelay    estdims     estshift    estvar      extract     fakeksp     
fft         fftmod      fftrot      fftshift    filter      flatten     
flip        fmac        homodyne    index       invert      itsense     
join        looklocker  lrmatrix    mandelbrot  mip         moba        
mobafit     nlinv       noise       normalize   nrmse       nufft       
ones        pattern     phantom     pics        pocsense    poisson     
pol2mask    poly        repmat      reshape     resize      rmfreq      
rof         roistat     rss         rtnlinv     sake        saxpy       
scale       sdot        show        signal      slice       spow        
sqpics      squeeze  

In [ ]:
! head -n2 image_out2.hdr
! head -n2 sensitivities.hdr
pics_reco = cfl.readcfl("image_out2")
pics_reco_sq = np.squeeze(pics_reco)
print(np.shape(pics_reco_sq))
vmax=0.5*np.max(np.abs(pics_reco))
# fig, axes = plt.subplots(figsize=(20,6), nrows=1, ncols=3, sharex=True, sharey=True)

plt.imshow(np.abs(pics_reco_sq[:,:,0]), cmap="gray", vmax=vmax)
plt.show()
# plt.imshow(np.abs(pics_reco_sq[:,:,1]), cmap="gray", vmax=vmax)
# plt.show()

head: cannot open 'image_out2.hdr' for reading: No such file or directory
head: cannot open 'sensitivities.hdr' for reading: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'image_out2.hdr'

### Plotting Fully sampled K space

In [ ]:
# ! bart fft -u -i 3 /content/data/kspace_ds kspace_fs_recon
# ! head -n2 /content/kspace_fs_recon.hdr

In [ ]:
# %%bash
# bart ecalib -r24 -m1 /content/data/data/kspace_fs coils_l
# FFT_FLAG=$(bart bitmask 0 1)
# COIL_FLAG=$(bart bitmask 3)
# bart fft -i -u $FFT_FLAG /content/data/data/kspace_fs coil_image
# bart fmac -C -s$COIL_FLAG coil_image coils_l image_ref


In [ ]:
# # ! head -n2 kspace_fs_recon.hdr
# ! head -n2 image_ref.hdr

In [ ]:
# pics_ref = cfl.readcfl("image_ref")
# print(np.shape(pics_ref))
# vmax=0.5*np.max(np.abs(pics_ref))
# plt.imshow(np.abs(pics_ref), cmap="gray", vmax=vmax)
# plt.show()


In [ ]:
# diff_map = np.abs(pics_reco_sq[:,:,0])/np.max(np.abs(pics_reco_sq[:,:,0])) - np.abs(pics_ref)/np.max(np.abs(pics_ref))
# plt.imshow(np.abs(diff_map), cmap="gray", vmax=1)
# plt.show()

### FastMri based downsampling and Sense Recon using Bart

In [ ]:
slice_kspace = volume_kspace[1]
print("Slice K-Space size: ",slice_kspace.shape)
reference=fastmri_to_bart(slice_kspace,None)
print("Slice K-Space size(in .cfl format): ",reference.shape)

In [ ]:
slice_kspace2 = T.to_tensor(slice_kspace)
print(slice_kspace2.dtype)

In [ ]:
from fastmri.data.subsample import RandomMaskFunc
mask_func = RandomMaskFunc(center_fractions=[0.04], accelerations=[2])  # Create the mask function object

In [ ]:
masked_kspace = T.apply_mask(slice_kspace2, mask_func) # Apply the mask to k-space

In [ ]:
type(masked_kspace)
print(masked_kspace[0].dtype)
print(masked_kspace[0].shape)
masked_kspace_npy = masked_kspace[0].numpy()
masked_kspace_npy.shape
masked_kspace_C_npy = masked_kspace_npy[:,:,:,0] + 1j* masked_kspace_npy[:,:,:,1]
masked_kspace_C_npy.shape

k_fastmri_ds = fastmri_to_bart(masked_kspace_C_npy, None)
k_fastmri_ds.shape


### Visualizing the Data

In [ ]:
sampled_path=buffer_path+"downsampled_fastmri"
reference_path=buffer_path+"fullspace_fastmri"
print(sampled_path)
print(reference_path)

cfl.writecfl(sampled_path,k_fastmri_ds )
cfl.writecfl(reference_path,reference )

In [ ]:
#converting np array to cfl
a = cfl.readcfl(reference_path)
type(a)
print(a.shape)
ref_img_path=cfl_path+"image_fs"
cfl.writecfl(ref_img_path,a )
b= cfl.writecfl(ref_img_path, a )

In [ ]:
# %%bash
# bart ecalib -r24 -m1 /content/data/data/kspace_ds_fastmri_new coils_l
# FFT_FLAG=$(bart bitmask 0 1)
# COIL_FLAG=$(bart bitmask 3)
# bart fft -i -u $FFT_FLAG /content/data/data/kspace_ds_fastmri_new coil_image
# bart fmac -C -s$COIL_FLAG coil_image coils_l image_ref_new_ds

In [ ]:
%%bash
bart ecalib -r24 -m1 /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/CFL/coils_l
FFT_FLAG=$(bart bitmask 0 1)
COIL_FLAG=$(bart bitmask 3)
bart fft -i -u $FFT_FLAG /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/CFL/coil_image
bart fmac -C -s$COIL_FLAG /content/drive/MyDrive/MRI/CFL/coil_image /content/drive/MyDrive/MRI/CFL/coils_l /content/drive/MyDrive/MRI/CFL/image_ds

In [ ]:
#Visualizing actual Image(reference Image)
ds_image_path=cfl_path + "image_ds"
pics_ds = cfl.readcfl(ds_image_path)
print(np.shape(pics_ds))
vmax=0.5*np.max(np.abs(pics_ds))
plt.imshow(np.abs(pics_ds), cmap="gray", vmax=vmax)
plt.show()

In [ ]:
# !bart ecalib /content/data/data/kspace_ds_fastmri_new sensitivities
# !bart pics -l1 -r0.001 /content/data/data/kspace_ds_fastmri_new sensitivities image_out_fmri

In [ ]:
#BART Image Reconstruction
!bart ecalib /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/BUFFER/sensitivities
!bart pics -l1 -r0.001 /content/drive/MyDrive/MRI/BUFFER/downsampled_fastmri /content/drive/MyDrive/MRI/BUFFER/sensitivities /content/drive/MyDrive/MRI/CFL/image_out_fmri

In [ ]:
#BART Reconstructed Image Visualization
reconstructed_image_path=cfl_path + "image_out_fmri"
pics_reco = cfl.readcfl(reconstructed_image_path)
vmax=0.5*np.max(np.abs(pics_reco))
pics_reco_sq = np.squeeze(pics_reco)
print(np.shape(pics_reco_sq))

# fig, axes = plt.subplots(figsize=(20,6), nrows=1, ncols=3, sharex=True, sharey=True)

plt.imshow(np.abs(pics_reco_sq[:,:,0]), cmap="gray", vmax=vmax)
plt.show()


In [ ]:
diff_map = np.abs(pics_reco_sq[:,:,0])/np.max(np.abs(pics_reco_sq[:,:,0])) - np.abs(pics_ref)/np.max(np.abs(pics_ref))
plt.imshow(np.abs(diff_map), cmap="gray", vmax=1)
plt.show()

In [ ]:
diff_map = np.abs(pics_ds)/np.max(np.abs(pics_ds)) - np.abs(pics_ref)/np.max(np.abs(pics_ref))
plt.imshow(np.abs(diff_map), cmap="gray", vmax=1)
plt.show()

In [ ]:
import os

# Remove an empty directory
try:
    os.rmdir(buffer_path)
    print(f"Directory '{buffer_path}' removed successfully.")
except OSError as e:
    print(f"Error: {e.strerror}")


### Plotting Downsampled K space

In [ ]:
! bart fft -u -i 3 /content/data/kspace_ds kspace_DS_recon
! head -n2 kspace_DS_recon.hdr

Loading cfl file /content/data/kspace_ds
 : No such file or directory
ERROR: head: cannot open 'kspace_DS_recon.hdr' for reading: No such file or directory


In [ ]:
%%bash
bart ecalib -r24 -m1 /content/data/kspace_ds coils_l
FFT_FLAG=$(bart bitmask 0 1)
COIL_FLAG=$(bart bitmask 3)
bart fft -i -u $FFT_FLAG /content/data/kspace_ds coil_image
bart fmac -C -s$COIL_FLAG coil_image coils_l image_DS


In [ ]:

pics_DS = cfl.readcfl("image_DS")
print(np.shape(pics_DS))
vmax=0.5*np.max(np.abs(pics_DS))
plt.imshow(np.abs(pics_DS), cmap="gray", vmax=vmax)
plt.show()

In [ ]:
diff_map = np.abs(pics_DS)/np.max(np.abs(pics_DS)) - np.abs(pics_ref)/np.max(np.abs(pics_ref))
plt.imshow(np.abs(diff_map), cmap="gray", vmax=1)
plt.show()

FASTMRI PARSING

In [ ]:
! head -n2 /content/data/data/kspace_fs_new.hdr

In [ ]:
%%bash
bart ecalib -r24 -m1 /content/data/data/kspace_fs_new coils_l
FFT_FLAG=$(bart bitmask 0 1)
COIL_FLAG=$(bart bitmask 3)
bart fft -i -u $FFT_FLAG /content/data/data/kspace_fs_new coil_image
bart fmac -C -s$COIL_FLAG coil_image coils_l image_ref_new

In [ ]:
pics_ref = cfl.readcfl("image_ref_new")
print(np.shape(pics_ref))
vmax=0.5*np.max(np.abs(pics_ref))
plt.imshow(np.abs(pics_ref), cmap="gray", vmax=vmax)
plt.show()

In [ ]:
# Get a list of all .h5 files in the directory
h5_files = glob.glob(os.path.join(h5_path, "*.h5"))
ll=[]

for h5_file in h5_files:
  temp=h5_file.replace(h5_path,"")
  temp=temp.replace(".h5","")
  ll.append(temp)

for index,tt in enumerate(ll):
  print(index,":\t", tt)
print("\nEnter the index of the file name from the above list: ")

ins = input()
h5_file_name=h5_path+ll[int(ins)]+".h5"

hf= h5py.File(h5_file_name)
print(h5_file_name)
print('Keys:', list(hf.keys()))

In [ ]:
volume_kspace = hf['kspace'][()]
print(volume_kspace.shape)

In [ ]:
# slice_no=1
# slice_kspace = volume_kspace[slice_no,:,:,:]
# type(slice_kspace)
# print(slice_kspace.shape)
# slice_kspace_format = np.transpose(slice_kspace, axes=[1,2,0])
# print(slice_kspace_format.shape)
# slice_kspace_format2= np.expand_dims(slice_kspace_format, axis=2)
# print(slice_kspace_format2.shape)

def fastmri_to_bart(data_kspace, slice_no):
    if slice_no == None:
        slice_kspace = data_kspace
    else:
        slice_kspace = data_kspace[slice_no,:,:,:]
    slice_kspace_format = np.transpose(slice_kspace, axes=[1,2,0])
    slice_kspace_format2=np.expand_dims(slice_kspace_format, axis=2)
    return slice_kspace_format2

slice_wrapper = fastmri_to_bart(volume_kspace,1)
error = slice_wrapper - slice_kspace_format2
print(np.unique(error))




In [ ]:
cfl_file_name=cfl_path+h5_file_name+"_"+str(slice_no)
cfl.writecfl(cfl_file_name, slice_kspace_format2 )

In [ ]:
fs_image_path=cfl_path + "image_ds"
pics_ref = cfl.readcfl(fs_image_path)

print(np.shape(pics_ref))
vmax=0.5*np.max(np.abs(pics_ref))
plt.imshow(np.abs(pics_ref), cmap="gray", vmax=vmax)
plt.show()